In [1]:
!pip install -q -U transformers
!pip install sentencepiece

!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... don

In [2]:
from datasets import load_dataset

In [3]:
# from accelerate import FullyShardedDataParallelPlugin, Accelerator
# from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

# fsdp_plugin = FullyShardedDataParallelPlugin(
#     state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
#     optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
# )

# accelerator = Accelerator(fsdp_plugin=fsdp_plugin)


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModelForSeq2SeqLM

model_id = "google/flan-t5-large"
#model_id = "openlm-research/open_llama_3b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# set tokenizer settings to save memory
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForSeq2SeqLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

## Metaphor Probabilities (zero-shot) for the pretrained model CODE

In [5]:
import numpy as np
from scipy.special import softmax
import pdb
import pandas as pd
import math
from typing import List
import random
import argparse
import torch


def sent_scoring(model_tokenizer, text, cuda, score_type="loss", output_attentions=False, length_normalize=False):
    model = model_tokenizer[0]
    tokenizer = model_tokenizer[1]
    assert model is not None
    assert tokenizer is not None
    encoded_text = tokenizer.encode(text)
    input_ids = torch.tensor(encoded_text).unsqueeze(0)
    if cuda:
        input_ids = input_ids.to('cuda')
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids, output_attentions=output_attentions)
    loss, logits = outputs[:2]

    sentence_prob = loss.item()
    if score_type == "prob":
        if length_normalize:
            mult = 2
        else:
            mult = len(encoded_text)

        sentence_prob = math.exp(-1.0 * loss * (mult - 1))

    if output_attentions:
        attn = outputs["attentions"]
        return sentence_prob, attn, input_ids

    return sentence_prob

def confusion_matrix(P_forward_1, P_forward_2, P_backward_1, P_backward_2):
    correct_forward = len(np.where(np.array(P_forward_1) >= 0.5)[0]) + len(np.where(np.array(P_forward_2) >=0.5)[0])
    wrong_forward = len(P_forward_1) + len(P_forward_2) - correct_forward

    correct_backward = len(np.where(np.array(P_backward_1) >= 0.5)[0]) + len(np.where(np.array(P_backward_2) >=0.5)[0])
    wrong_backward = len(P_backward_1) + len(P_backward_2) - correct_backward

    print("correct forward", correct_forward, "wrong forward", wrong_forward, "correct backward", correct_backward, "wrong_backward", wrong_backward)

def evaluate_model(model, tokenizer, test_set, middle_phrase="", use_prefix=0, verbose=True, score_type="prob", use_cuda=False, return_acc=False) -> tuple:
    preds = []
    labels = []
    x_1 = []
    x_2 = []
    y_1 = []
    y_2 = []
    P_x_1 = []
    P_x_2 = []
    P_y_1 = []
    P_y_2 = []
    P_x_1_y_1 = []
    P_x_1_y_2 = []
    P_x_2_y_1 = []
    P_x_2_y_2 = []
    P_x_1_correct = []
    P_x_2_correct = []
    P_y_1_correct = []
    P_y_2_correct = []
    correct = 0

    for i, metaphor_data in enumerate(test_set):
        ctx, p1, p2 = metaphor_data["startphrase"], metaphor_data["ending1"], metaphor_data["ending2"]
        labels.append(int(metaphor_data["labels"]))
        if use_prefix > 0:
            prefix_prompt = select_prefix_prompts(prompt_file, use_prefix) if use_prefix else ""
        else:
            prefix_prompt = ""

        sent1 = prefix_prompt + ctx + ". " + middle_phrase + p1 + "."
        sent2 = prefix_prompt + ctx + ". " + middle_phrase + p2 + "."

        score1 = sent_scoring((model, tokenizer), sent1, use_cuda, score_type=score_type)
        score2 = sent_scoring((model, tokenizer), sent2, use_cuda, score_type=score_type)

        if score_type == "loss":
            pred = 0 if score1 < score2 else 1
        else:
            pred = 1 if score1 < score2 else 0

        pred_sent = sent1 if pred == 0 else sent2

        if i % 2 == 0:
            x_1.append(ctx)
            x_1_score = sent_scoring((model, tokenizer), ctx + ".", use_cuda, score_type=score_type)
            P_x_1.append(x_1_score)
            y_1.append(p1)
            y_2.append(p2)
            y1_score = sent_scoring((model, tokenizer), p1 + ".", use_cuda, score_type=score_type)
            y2_score = sent_scoring((model, tokenizer), p2 + ".", use_cuda, score_type=score_type)
            P_y_1.append(y1_score)
            P_y_2.append(y2_score)

            P_x_1_y_1.append(score1)
            P_x_1_y_2.append(score2)
            P_x_1_correct.append(score1/(score1 + score2))

        else:
            x_2.append(ctx)
            x_2_score = sent_scoring((model, tokenizer), ctx + ".", use_cuda, score_type=score_type)
            P_x_2.append(x_2_score)
            P_x_2_y_1.append(score1)
            P_x_2_y_2.append(score2)
            P_x_2_correct.append(score2/(score1 + score2))

            P_y_1_correct.append(P_x_1_y_1[-1]/(P_x_1_y_1[-1] + score1))
            P_y_2_correct.append(score2/(P_x_1_y_2[-1] + score2))

        if verbose:
            print(f"Q: {ctx}: 1. {p1} 2. {p2}")
            print(f"model says '{pred_sent}' is more likely")
            print("\n")
        if pred == metaphor_data["labels"]:
            correct += 1
        preds.append(pred)

    cols = {"x_1": x_1, "x_2": x_2, "y_1": y_1, "y_2": y_2, "P(x_1)": P_x_1, "P(x_2)": P_x_2, "P(y_1)": P_y_1, "P(y_2)": P_y_2,
        "P(x_1, y_1)": P_x_1_y_1, "P(x_1, y_2)": P_x_1_y_2, "P(x_2, y_1)": P_x_2_y_1, "P(x_2, y_2)": P_x_2_y_2,
        "P(y_1|x_1)": P_x_1_correct, "P(y_2|x_2)": P_x_2_correct, "P(x_1|y_1)": P_y_1_correct, "P(x_2|y_2)": P_y_2_correct}
    out_df = pd.DataFrame(cols)

    if return_acc:
        return correct/len(preds), out_df, preds, labels

    return out_df, preds, labels

def compute_stats(total_df: pd.DataFrame, all_preds: List, all_labels: List) -> None:
    print("overall accuracy: ")
    acc = len(np.where(np.array(all_preds) == np.array(all_labels))[0])/len(all_labels)
    print(acc)
    print("confusion matrix: ")
    confusion_matrix(list(total_df["P(y_1|x_1)"]), list(total_df["P(y_2|x_2)"]), list(total_df["P(x_1|y_1)"]), list(total_df["P(x_2|y_2)"]))

    x = {
        "P_forward_1": len(list(total_df["P(y_1|x_1)"])),
        "P_forward_2": len(list(total_df["P(y_2|x_2)"])),
        "P_backward_1":len(list(total_df["P(x_1|y_1)"])),
        "P_backward_2":len(list(total_df["P(x_2|y_2)"])),
        "accuracy": acc
    }
    return x

In [6]:
dataset = load_dataset("nightingal3/fig-qa")
dataset['validation']

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['startphrase', 'ending1', 'ending2', 'labels', 'valid'],
    num_rows: 1094
})

# Run Prompts

In [7]:
middle_phrases = [
'That is to put it mildly',
 "Let's face it",
 'Essentially speaking',
 'So to speak',
 'All things considered',
 'This is to say',
 '...',
 '=> ',
 'To put it another way',
 'To put it bluntly',
 'In other words',
 'To rephrase,',
 'Moreover',
 'how',
 'explain more',
 'Furthermore',
 'In conclusion',
 'more',
 'in other words',
 ""
]

In [8]:
subsubset_test_dataset = dataset['validation'].select(range(400))

result_middlephrase = {}
for middle in middle_phrases:
  md = middle + " "
  out_df, preds, labels = evaluate_model(model, tokenizer, subsubset_test_dataset, use_prefix=0, middle_phrase = md)
  res = compute_stats(out_df, preds, labels)
  result_middlephrase[middle] = res


Streaming output truncated to the last 5000 lines.
Q: Your wit has the sharpness of a blade: 1. Your wit is clever. 2. Your wit is dull and not clever.
model says 'Your wit has the sharpness of a blade. In conclusion Your wit is dull and not clever..' is more likely


Q: Your wit has the sharpness of a stone: 1. Your wit is clever. 2. Your wit is dull and not clever.
model says 'Your wit has the sharpness of a stone. In conclusion Your wit is dull and not clever..' is more likely


Q: She had all the grace and elegance of a ballerina: 1. She is very graceful and elegant. 2. She is not graceful or elegant, and in fact is cumbersome and unsophisticated.
model says 'She had all the grace and elegance of a ballerina. In conclusion She is very graceful and elegant..' is more likely


Q: She had all the grace and elegance of a dump truck: 1. She is very graceful and elegant. 2. She is not graceful or elegant, and in fact is cumbersome and unsophisticated.
model says 'She had all the grace an

In [9]:
# BASE with empty prompt
result_middlephrase

{'That is to put it mildly': {'P_forward_1': 200,
  'P_forward_2': 200,
  'P_backward_1': 200,
  'P_backward_2': 200,
  'accuracy': 0.61},
 "Let's face it": {'P_forward_1': 200,
  'P_forward_2': 200,
  'P_backward_1': 200,
  'P_backward_2': 200,
  'accuracy': 0.645},
 'Essentially speaking': {'P_forward_1': 200,
  'P_forward_2': 200,
  'P_backward_1': 200,
  'P_backward_2': 200,
  'accuracy': 0.6275},
 'So to speak': {'P_forward_1': 200,
  'P_forward_2': 200,
  'P_backward_1': 200,
  'P_backward_2': 200,
  'accuracy': 0.625},
 'All things considered': {'P_forward_1': 200,
  'P_forward_2': 200,
  'P_backward_1': 200,
  'P_backward_2': 200,
  'accuracy': 0.6175},
 'This is to say': {'P_forward_1': 200,
  'P_forward_2': 200,
  'P_backward_1': 200,
  'P_backward_2': 200,
  'accuracy': 0.675},
 '...': {'P_forward_1': 200,
  'P_forward_2': 200,
  'P_backward_1': 200,
  'P_backward_2': 200,
  'accuracy': 0.575},
 '=> ': {'P_forward_1': 200,
  'P_forward_2': 200,
  'P_backward_1': 200,
  'P_ba

In [10]:
max_accuracy = max(result_middlephrase, key=lambda x: result_middlephrase[x]['accuracy'])
max_accuracy_value = result_middlephrase[max_accuracy]['accuracy']

# Find all prompts with the maximum accuracy value
prompts_with_max_accuracy = [index for index, entry in result_middlephrase.items() if entry['accuracy'] == max_accuracy_value]

print(f"The maximum accuracy is {max_accuracy_value} at index/indices {', '.join(map(str, prompts_with_max_accuracy))}")

The maximum accuracy is 0.675 at index/indices This is to say


In [11]:
# Log results
result_middlephrase

{'That is to put it mildly': {'P_forward_1': 200,
  'P_forward_2': 200,
  'P_backward_1': 200,
  'P_backward_2': 200,
  'accuracy': 0.61},
 "Let's face it": {'P_forward_1': 200,
  'P_forward_2': 200,
  'P_backward_1': 200,
  'P_backward_2': 200,
  'accuracy': 0.645},
 'Essentially speaking': {'P_forward_1': 200,
  'P_forward_2': 200,
  'P_backward_1': 200,
  'P_backward_2': 200,
  'accuracy': 0.6275},
 'So to speak': {'P_forward_1': 200,
  'P_forward_2': 200,
  'P_backward_1': 200,
  'P_backward_2': 200,
  'accuracy': 0.625},
 'All things considered': {'P_forward_1': 200,
  'P_forward_2': 200,
  'P_backward_1': 200,
  'P_backward_2': 200,
  'accuracy': 0.6175},
 'This is to say': {'P_forward_1': 200,
  'P_forward_2': 200,
  'P_backward_1': 200,
  'P_backward_2': 200,
  'accuracy': 0.675},
 '...': {'P_forward_1': 200,
  'P_forward_2': 200,
  'P_backward_1': 200,
  'P_backward_2': 200,
  'accuracy': 0.575},
 '=> ': {'P_forward_1': 200,
  'P_forward_2': 200,
  'P_backward_1': 200,
  'P_ba

In [12]:
#  Print top Results
result_list = [[k,v["accuracy"]] for k,v in result_middlephrase.items()]
result_df = pd.DataFrame(result_list, columns=["prompt", "accuracy"])
sorted_df = result_df.sort_values(by='accuracy', ascending=False)
sorted_df

,prompt,accuracy
5,This is to say,0.6750
7,=>,0.6650
1,Let's face it,0.6450
2,Essentially speaking,0.6275
3,So to speak,0.6250
4,All things considered,0.6175
19,,0.6150
17,more,0.6100
0,That is to put it mildly,0.6100
16,In conclusion,0.6075
